import statements

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

import numpy as np
import time

parameters

In [ ]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Image preprocessing (+ image augmentation)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the desired image size
image_size = (256, 256)

# Define a function to preprocess and augment your dataset
def preprocess_image(image):
    # Resize the image to the desired size
    image = tf.image.resize(image, image_size)
    
    # Normalize pixel values to the range [-1, 1]
    image = (image - 127.5) / 127.5
    
    return image

# Create a data generator for your dataset with extended data augmentation
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_image,
    horizontal_flip=True,      # Flip images horizontally
    rotation_range=20,         # Rotate images by up to 20 degrees
    width_shift_range=0.1,     # Shift images horizontally by up to 10% of the width
    height_shift_range=0.1,    # Shift images vertically by up to 10% of the height
    shear_range=0.2,           # Shear transformations
    zoom_range=0.2,            # Zoom in/out
    channel_shift_range=0.1,   # Random channel shifts
    brightness_range=[0.7, 1.3]  # Adjust brightness
)

# Load your dataset (assuming it's organized in a directory structure)
dataset_path = '/training_images'  # path of dataset
batch_size = 32  # Adjust as needed

# Create a TensorFlow data generator
data_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None,  # Since you're doing style transfer, you don't need class labels
    shuffle=True
)


generator/descriminator

In [ ]:
# Define the style transfer generator with a pretrained VGG16 encoder
def build_style_transfer_generator(input_shape, vgg_encoder, latent_dim):
    # Freeze the VGG encoder layers to prevent them from being updated
    vgg_encoder.trainable = False
    
    # Decoder
    decoder = tf.keras.Sequential([
        # Convolutional layers with upsampling
        tf.keras.layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu'),
        tf.keras.layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu'),
        tf.keras.layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', activation='relu'),
        tf.keras.layers.Conv2DTranspose(3, (3, 3), strides=(2, 2), padding='same', activation='tanh')  # Output layer
    ])
    
    # Define the input layer
    input_image = tf.keras.layers.Input(shape=input_shape)
    
    # Encode the input image using the pretrained VGG model
    encoded = vgg_encoder(input_image)
    
    # Decode to get the stylized output
    stylized_output = decoder(encoded)
    
    # Create the generator model
    generator = tf.keras.Model(inputs=input_image, outputs=stylized_output)
    
    return generator

# Define the style evaluation discriminator
def build_style_evaluation_discriminator(input_shape):
    model = tf.keras.Sequential([
        # Convolutional layers with LeakyReLU activations
        tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.2)),
        tf.keras.layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.2)),
        tf.keras.layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1)  # Output layer for binary classification
    ])
    
    # Define the input layer
    input_image = tf.keras.layers.Input(shape=input_shape)
    
    # Get the discriminator's evaluation
    evaluation = model(input_image)
    
    # Create the discriminator model
    discriminator = tf.keras.Model(inputs=input_image, outputs=evaluation)
    
    return discriminator

# Define the input shape and latent dimension
input_shape = (256, 256, 3)
latent_dim = 100  # Adjust as needed

# Create a VGG16 model for use as the encoder
vgg_encoder = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)

# Build the style transfer generator and discriminator
generator = build_style_transfer_generator(input_shape, vgg_encoder, latent_dim)
discriminator = build_style_evaluation_discriminator(input_shape)

# Compile the discriminator
discriminator.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002)
)

# Print model summaries for reference
generator.summary()
discriminator.summary()